## The ClusterFewShot Optimizer

In this notebook, I present the ClusterFewShot Optimizer. I demonstrate how to use it and evaluate it against other optimizers on two DSPy programs: Basic RAG and Simplified Baleen (taken from intro.ipynb.)

In [ ]:
%pip install -r requirements.txt
%pip install fastembed

In [65]:
import dsp
from dsp.modules.sentence_vectorizer import FastEmbedVectorizer
import dspy
import openai
import os
import importlib
from datasets import load_dataset
from dspy.evaluate.evaluate import Evaluate

In [ ]:
os.environ["OPENAI_API_KEY"] = ""
os.environ["OPENAI_API_BASE"] = ""

#### Program 1: Basic RAG

In [28]:
lm = dspy.LM("openai/gpt-4o-mini")
lm("testing")

# [{"role": "user", "content": ___}]
# lm(messages=[{"role": "user", "content": "Hello, how old are you?"}])

["It looks like you're testing the system. How can I assist you today?"]

In [29]:
colbertv2_wiki17_abstracts = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')

dspy.settings.configure(lm=lm, rm=colbertv2_wiki17_abstracts)

In [34]:
from dspy.datasets import HotPotQA

# Load the dataset.
dataset = HotPotQA(train_seed=1, train_size=20, eval_seed=2023, dev_size=50, test_size=0)

# Tell DSPy that the 'question' field is the input. Any other fields are labels and/or metadata.
trainset = [x.with_inputs('question') for x in dataset.train]
devset = [x.with_inputs('question') for x in dataset.dev]

len(trainset), len(devset)

(20, 50)

In [7]:
trainset[0]

Example({'question': 'At My Window was released by which American singer-songwriter?', 'answer': 'John Townes Van Zandt'}) (input_keys={'question'})

In [35]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()

        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)

    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

In [36]:
importlib.reload(dspy)
importlib.reload(dspy.teleprompt)
importlib.reload(dspy.teleprompt.utils)
importlib.reload(dspy.teleprompt.cluster_fewshot)

<module 'dspy.teleprompt.cluster_fewshot' from 'c:\\Users\\Hamza\\dspy\\dspy\\teleprompt\\cluster_fewshot.py'>

In [37]:
from dspy.teleprompt import LabeledFewShot
from dspy.teleprompt import BootstrapFewShot
from dspy.teleprompt import ClusterFewShot

def validate_context_and_answer(example, pred, trace=None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    return answer_EM and answer_PM

vanilla_teleprompter = LabeledFewShot(k=6)
bootstrap_teleprompter = BootstrapFewShot(max_labeled_demos=6, metric=validate_context_and_answer)
cluster_teleprompter = ClusterFewShot(num_labeled_demos=6)

compiled_rag_vanilla = vanilla_teleprompter.compile(RAG(), trainset=trainset)
compiled_rag_bootstrap = bootstrap_teleprompter.compile(RAG(), trainset=trainset)
compiled_rag_cluster = cluster_teleprompter.compile(RAG(), trainset=trainset)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]


 55%|█████▌    | 11/20 [00:00<00:00, 329.05it/s]


Bootstrapped 4 full traces after 11 examples for up to 1 rounds, amounting to 11 attempts.


In [ ]:
lm.inspect_history(n=1)

###### In the next 2 cells, we check which demonstrations were selected by LabeledFewShot and ClusterFewShot. As expected, they selected different examples. Notice that ClusterFewShot did not select both the two demos that have answers "Bill Murray" and "Bill Paxton", unlike LabeledFewShot (vanilla), since these similar demos were clustered together (as we will soon see).

In [9]:
for name, parameter in compiled_rag_vanilla.named_predictors():
    print(name)
    for demo in parameter.demos:
        print(demo)
    print()

generate_answer
Example({'question': 'Samantha Cristoforetti and Mark Shuttleworth are both best known for being first in their field to go where? ', 'answer': 'space'}) (input_keys={'question'})
Example({'question': 'Having the combination of excellent foot speed and bat speed helped Eric Davis, create what kind of outfield for the Los Angeles Dodgers? ', 'answer': '"Outfield of Dreams"'}) (input_keys={'question'})
Example({'question': 'which  American actor was Candace Kita  guest starred with ', 'answer': 'Bill Murray'}) (input_keys={'question'})
Example({'question': 'Tombstone stared an actor born May 17, 1955 known as who?', 'answer': 'Bill Paxton'}) (input_keys={'question'})
Example({'question': 'The Organisation that allows a community to influence their operation or use and to enjoy the benefits arisingwas founded in what year?', 'answer': '2010'}) (input_keys={'question'})
Example({'question': 'Which is taller, the Empire State Building or the Bank of America Tower?', 'answer'

In [10]:
for name, parameter in compiled_rag_cluster.named_predictors():
    print(name)
    for demo in parameter.demos:
        print(demo)
    print()

generate_answer
Example({'question': 'Which American actress who made their film debut in the 1995 teen drama "Kids" was the co-founder of Voto Latino?', 'answer': 'Rosario Dawson'}) (input_keys={'question'})
Example({'question': 'Which company distributed this 1977 American animated film produced by Walt Disney Productions for which Sherman Brothers wrote songs?', 'answer': 'Buena Vista Distribution'}) (input_keys={'question'})
Example({'question': 'Which Pakistani cricket umpire who won 3 consecutive ICC umpire of the year awards in 2009, 2010, and 2011 will be in the ICC World Twenty20?', 'answer': 'Aleem Sarwar Dar'}) (input_keys={'question'})
Example({'question': 'In what year was the club founded that played Manchester City in the 1972 FA Charity Shield', 'answer': '1874'}) (input_keys={'question'})
Example({'question': 'which  American actor was Candace Kita  guest starred with ', 'answer': 'Bill Murray'}) (input_keys={'question'})
Example({'question': 'At My Window was released

###### Let's look at the clusters created by ClusterFewShot.

In [11]:
for name, parameter in compiled_rag_cluster.named_predictors():
    for key, val in parameter.cluster_assignments.items():
        print(key)
        for demo in val:
            print(demo)
        print()

7


Example({'question': 'At My Window was released by which American singer-songwriter?', 'answer': 'John Townes Van Zandt'}) (input_keys={'question'})
Example({'question': 'Which magazine has published articles by Scott Shaw, Tae Kwon Do Times or Southwest Art?', 'answer': 'Tae Kwon Do Times'}) (input_keys={'question'})
Example({'question': 'Having the combination of excellent foot speed and bat speed helped Eric Davis, create what kind of outfield for the Los Angeles Dodgers? ', 'answer': '"Outfield of Dreams"'}) (input_keys={'question'})
Example({'question': 'This American guitarist best known for her work with the Iron Maidens is an ancestor of a composer who was known as what?', 'answer': 'The Waltz King'}) (input_keys={'question'})

4
Example({'question': 'which  American actor was Candace Kita  guest starred with ', 'answer': 'Bill Murray'}) (input_keys={'question'})
Example({'question': 'Tombstone stared an actor born May 17, 1955 known as who?', 'answer': 'Bill Paxton'}) (input_

###### It looks like nine clusters were created. Inspecting the clusters closely, it seems like some choices make sense, for example, clustering two questions about an actress together in cluster 0 and two questions about an actor (both named Bill) together in cluster 4. However, one could argue that these two clusters should have been combined into one. Also, it's not clear why cluster 3 only contains 2 out of the 3 questions that ask about a year, the other one being assigned to cluster 6.

### Evaluation

###### Now let's evaluate ClusterFewShot's performance against LabeledFewShot and BootstrapFewShot!

In [38]:
evaluate_on_hotpotqa = Evaluate(devset=devset, num_threads=1, display_progress=True, display_table=5)

metric = dspy.evaluate.answer_exact_match
evaluate_on_hotpotqa(compiled_rag_vanilla, metric=metric)

Average Metric: 26 / 50  (52.0): 100%|██████████| 50/50 [00:00<00:00, 215.72it/s]


,question,example_answer,gold_titles,context,pred_answer,answer_exact_match
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Qionghai', 'Cangzhou'}","['Cangzhou | Cangzhou () is a prefecture-level city in eastern Hebei province, People\'s Republic of China. At the 2010 census, Cangzhou\'s built-up (""or metro"") area...","No, only Cangzhou is.",
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017–18 Pittsburgh Penguins season', '2017 NHL Expansion Draft'}",['2017–18 Pittsburgh Penguins season | The 2017–18 Pittsburgh Penguins season will be the 51st season for the National Hockey League ice hockey team that was...,National Hockey League,✔️ [True]
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'2006–07 Detroit Red Wings season', 'Steve Yzerman'}","['Steve Yzerman | Stephen Gregory ""Steve"" Yzerman ( ; born May 9, 1965) is a Canadian retired professional ice hockey player and current general manager...",Steve Yzerman,✔️ [True]
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Collegiate Church', 'Crichton Castle'}","[""Crichton Collegiate Church | Crichton Collegiate Church is situated about 0.6 mi south west of the hamlet of Crichton in Midlothian, Scotland. Crichton itself is...",River Tyne,✔️ [True]
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Ealhswith', 'Æthelweard (son of Alfred)'}","[""Æthelweard of East Anglia | Æthelweard (died 854) was a 9th-century king of East Anglia, the long-lived Anglo-Saxon kingdom which today includes the English counties...",King Alfred the Great,✔️ [True]


52.0

In [39]:
evaluate_on_hotpotqa(compiled_rag_bootstrap, metric=metric)

Average Metric: 29 / 50  (58.0): 100%|██████████| 50/50 [00:00<00:00, 232.61it/s]


,question,example_answer,gold_titles,context,pred_answer,answer_exact_match
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Qionghai', 'Cangzhou'}","['Cangzhou | Cangzhou () is a prefecture-level city in eastern Hebei province, People\'s Republic of China. At the 2010 census, Cangzhou\'s built-up (""or metro"") area...",No,✔️ [True]
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017–18 Pittsburgh Penguins season', '2017 NHL Expansion Draft'}",['2017–18 Pittsburgh Penguins season | The 2017–18 Pittsburgh Penguins season will be the 51st season for the National Hockey League ice hockey team that was...,National Hockey League,✔️ [True]
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'2006–07 Detroit Red Wings season', 'Steve Yzerman'}","['Steve Yzerman | Stephen Gregory ""Steve"" Yzerman ( ; born May 9, 1965) is a Canadian retired professional ice hockey player and current general manager...",Steve Yzerman,✔️ [True]
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Collegiate Church', 'Crichton Castle'}","[""Crichton Collegiate Church | Crichton Collegiate Church is situated about 0.6 mi south west of the hamlet of Crichton in Midlothian, Scotland. Crichton itself is...",River Tyne,✔️ [True]
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Ealhswith', 'Æthelweard (son of Alfred)'}","[""Æthelweard of East Anglia | Æthelweard (died 854) was a 9th-century king of East Anglia, the long-lived Anglo-Saxon kingdom which today includes the English counties...",King Alfred the Great,✔️ [True]


58.0

In [40]:
evaluate_on_hotpotqa(compiled_rag_cluster, metric=metric)

Average Metric: 30 / 50  (60.0): 100%|██████████| 50/50 [01:29<00:00,  1.79s/it]


,question,example_answer,gold_titles,context,pred_answer,answer_exact_match
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Qionghai', 'Cangzhou'}","['Cangzhou | Cangzhou () is a prefecture-level city in eastern Hebei province, People\'s Republic of China. At the 2010 census, Cangzhou\'s built-up (""or metro"") area...",No,✔️ [True]
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017–18 Pittsburgh Penguins season', '2017 NHL Expansion Draft'}",['2017–18 Pittsburgh Penguins season | The 2017–18 Pittsburgh Penguins season will be the 51st season for the National Hockey League ice hockey team that was...,National Hockey League,✔️ [True]
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'2006–07 Detroit Red Wings season', 'Steve Yzerman'}","['Steve Yzerman | Stephen Gregory ""Steve"" Yzerman ( ; born May 9, 1965) is a Canadian retired professional ice hockey player and current general manager...",Steve Yzerman,✔️ [True]
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Collegiate Church', 'Crichton Castle'}","[""Crichton Collegiate Church | Crichton Collegiate Church is situated about 0.6 mi south west of the hamlet of Crichton in Midlothian, Scotland. Crichton itself is...",River Tyne,✔️ [True]
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Ealhswith', 'Æthelweard (son of Alfred)'}","[""Æthelweard of East Anglia | Æthelweard (died 854) was a 9th-century king of East Anglia, the long-lived Anglo-Saxon kingdom which today includes the English counties...",King Alfred the Great,✔️ [True]


60.0

###### Looks like compiled_rag_cluster (60.0) outperforms compiled_rag_vanilla (58.0) and compiled_rag_bootstrap (52.0)!

In [55]:
def gold_passages_retrieved(example, pred, trace=None):
    gold_titles = set(map(dspy.evaluate.normalize_text, example['gold_titles']))
    found_titles = set(map(dspy.evaluate.normalize_text, [c.split(' | ')[0] for c in pred.context]))

    return gold_titles.issubset(found_titles)

compiled_rag_vanilla_retrieval_score = evaluate_on_hotpotqa(compiled_rag_vanilla, metric=gold_passages_retrieved)
compiled_rag_bootstrap_retrieval_score = evaluate_on_hotpotqa(compiled_rag_bootstrap, metric=gold_passages_retrieved)
compiled_rag_cluster_retrieval_score = evaluate_on_hotpotqa(compiled_rag_cluster, metric=gold_passages_retrieved)

Average Metric: 13 / 50  (26.0): 100%|██████████| 50/50 [00:00<00:00, 124.63it/s]


,question,example_answer,gold_titles,context,pred_answer,gold_passages_retrieved
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Qionghai', 'Cangzhou'}","['Cangzhou | Cangzhou () is a prefecture-level city in eastern Hebei province, People\'s Republic of China. At the 2010 census, Cangzhou\'s built-up (""or metro"") area...","No, only Cangzhou is.",
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017–18 Pittsburgh Penguins season', '2017 NHL Expansion Draft'}",['2017–18 Pittsburgh Penguins season | The 2017–18 Pittsburgh Penguins season will be the 51st season for the National Hockey League ice hockey team that was...,National Hockey League,✔️ [True]
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'2006–07 Detroit Red Wings season', 'Steve Yzerman'}","['Steve Yzerman | Stephen Gregory ""Steve"" Yzerman ( ; born May 9, 1965) is a Canadian retired professional ice hockey player and current general manager...",Steve Yzerman,✔️ [True]
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Collegiate Church', 'Crichton Castle'}","[""Crichton Collegiate Church | Crichton Collegiate Church is situated about 0.6 mi south west of the hamlet of Crichton in Midlothian, Scotland. Crichton itself is...",River Tyne,✔️ [True]
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Ealhswith', 'Æthelweard (son of Alfred)'}","[""Æthelweard of East Anglia | Æthelweard (died 854) was a 9th-century king of East Anglia, the long-lived Anglo-Saxon kingdom which today includes the English counties...",King Alfred the Great,


Average Metric: 13 / 50  (26.0): 100%|██████████| 50/50 [00:00<00:00, 183.12it/s]


,question,example_answer,gold_titles,context,pred_answer,gold_passages_retrieved
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Qionghai', 'Cangzhou'}","['Cangzhou | Cangzhou () is a prefecture-level city in eastern Hebei province, People\'s Republic of China. At the 2010 census, Cangzhou\'s built-up (""or metro"") area...",No,
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017–18 Pittsburgh Penguins season', '2017 NHL Expansion Draft'}",['2017–18 Pittsburgh Penguins season | The 2017–18 Pittsburgh Penguins season will be the 51st season for the National Hockey League ice hockey team that was...,National Hockey League,✔️ [True]
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'2006–07 Detroit Red Wings season', 'Steve Yzerman'}","['Steve Yzerman | Stephen Gregory ""Steve"" Yzerman ( ; born May 9, 1965) is a Canadian retired professional ice hockey player and current general manager...",Steve Yzerman,✔️ [True]
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Collegiate Church', 'Crichton Castle'}","[""Crichton Collegiate Church | Crichton Collegiate Church is situated about 0.6 mi south west of the hamlet of Crichton in Midlothian, Scotland. Crichton itself is...",River Tyne,✔️ [True]
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Ealhswith', 'Æthelweard (son of Alfred)'}","[""Æthelweard of East Anglia | Æthelweard (died 854) was a 9th-century king of East Anglia, the long-lived Anglo-Saxon kingdom which today includes the English counties...",King Alfred the Great,


Average Metric: 13 / 50  (26.0): 100%|██████████| 50/50 [00:00<00:00, 134.87it/s]


,question,example_answer,gold_titles,context,pred_answer,gold_passages_retrieved
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Qionghai', 'Cangzhou'}","['Cangzhou | Cangzhou () is a prefecture-level city in eastern Hebei province, People\'s Republic of China. At the 2010 census, Cangzhou\'s built-up (""or metro"") area...",No,
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017–18 Pittsburgh Penguins season', '2017 NHL Expansion Draft'}",['2017–18 Pittsburgh Penguins season | The 2017–18 Pittsburgh Penguins season will be the 51st season for the National Hockey League ice hockey team that was...,National Hockey League,✔️ [True]
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'2006–07 Detroit Red Wings season', 'Steve Yzerman'}","['Steve Yzerman | Stephen Gregory ""Steve"" Yzerman ( ; born May 9, 1965) is a Canadian retired professional ice hockey player and current general manager...",Steve Yzerman,✔️ [True]
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Collegiate Church', 'Crichton Castle'}","[""Crichton Collegiate Church | Crichton Collegiate Church is situated about 0.6 mi south west of the hamlet of Crichton in Midlothian, Scotland. Crichton itself is...",River Tyne,✔️ [True]
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Ealhswith', 'Æthelweard (son of Alfred)'}","[""Æthelweard of East Anglia | Æthelweard (died 854) was a 9th-century king of East Anglia, the long-lived Anglo-Saxon kingdom which today includes the English counties...",King Alfred the Great,


###### In terms of retrieval, it seems like all optimizers have the same performance (26.0).

#### Program 2: Simplified Baleen

###### Let's repeat the same process but for a more complex program: Simplified Baleen.

In [48]:
class GenerateSearchQuery(dspy.Signature):
    """Write a simple search query that will help answer a complex question."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    query = dspy.OutputField()

from dsp.utils import deduplicate

class SimplifiedBaleen(dspy.Module):
    def __init__(self, passages_per_hop=3, max_hops=2):
        super().__init__()

        self.generate_query = [dspy.ChainOfThought(GenerateSearchQuery) for _ in range(max_hops)]
        self.retrieve = dspy.Retrieve(k=passages_per_hop)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
        self.max_hops = max_hops

    def forward(self, question):
        context = []

        for hop in range(self.max_hops):
            query = self.generate_query[hop](context=context, question=question).query
            passages = self.retrieve(query).passages
            context = deduplicate(context + passages)

        pred = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=pred.answer)

In [49]:
def validate_context_and_answer_and_hops(example, pred, trace=None):
    if not dspy.evaluate.answer_exact_match(example, pred): return False
    if not dspy.evaluate.answer_passage_match(example, pred): return False

    hops = [example.question] + [outputs.query for *_, outputs in trace if 'query' in outputs]

    if max([len(h) for h in hops]) > 100: return False
    if any(dspy.evaluate.answer_exact_match_str(hops[idx], hops[:idx], frac=0.8) for idx in range(2, len(hops))): return False

    return True

In [50]:
vanilla_teleprompter = LabeledFewShot()
bootstrap_teleprompter = BootstrapFewShot(metric=validate_context_and_answer_and_hops)
cluster_teleprompter = ClusterFewShot()

compiled_baleen_vanilla = vanilla_teleprompter.compile(SimplifiedBaleen(), trainset=trainset)
compiled_baleen_bootstrap = bootstrap_teleprompter.compile(SimplifiedBaleen(), teacher=SimplifiedBaleen(passages_per_hop=2), trainset=trainset)
compiled_baleen_cluster = cluster_teleprompter.compile(SimplifiedBaleen(), trainset=trainset)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 20/20 [00:01<00:00, 10.79it/s]


Bootstrapped 3 full traces after 19 examples for up to 1 rounds, amounting to 20 attempts.


In [51]:
for name, parameter in compiled_baleen_bootstrap.named_predictors():
    print(name)
    for demo in parameter.demos:
        print(demo)
    print()

generate_query[0]
Example({'augmented': True, 'context': [], 'question': 'In what year was the club founded that played Manchester City in the 1972 FA Charity Shield', 'reasoning': 'To answer the question, I need to identify the club that played against Manchester City in the 1972 FA Charity Shield. Once I have the name of that club, I can then look up the year it was founded.', 'query': '"1972 FA Charity Shield Manchester City opponent club founded year"'}) (input_keys=None)
Example({'augmented': True, 'context': [], 'question': '"Everything Has Changed" is a song from an album released under which record label ?', 'reasoning': 'To answer the question about the record label for the song "Everything Has Changed," I need to identify the album it belongs to and then find out which record label released that album.', 'query': '"Everything Has Changed" song album record label'}) (input_keys=None)
Example({'augmented': True, 'context': [], 'question': 'Who is older, Aleksandr Danilovich Ale

In [52]:
for name, parameter in compiled_baleen_cluster.named_predictors():
    print(name)
    for demo in parameter.demos:
        print(demo)
    print()

generate_query[0]
Example({'question': 'Which American actress who made their film debut in the 1995 teen drama "Kids" was the co-founder of Voto Latino?', 'answer': 'Rosario Dawson'}) (input_keys={'question'})
Example({'question': 'Who acted in the shot film The Shore and is also the youngest actress ever to play Ophelia in a Royal Shakespeare Company production of "Hamlet." ?', 'answer': 'Kerry Condon'}) (input_keys={'question'})
Example({'question': 'Which company distributed this 1977 American animated film produced by Walt Disney Productions for which Sherman Brothers wrote songs?', 'answer': 'Buena Vista Distribution'}) (input_keys={'question'})
Example({'question': 'Samantha Cristoforetti and Mark Shuttleworth are both best known for being first in their field to go where? ', 'answer': 'space'}) (input_keys={'question'})
Example({'question': 'Which Pakistani cricket umpire who won 3 consecutive ICC umpire of the year awards in 2009, 2010, and 2011 will be in the ICC World Twenty

### Evaluation

###### We will evaluate ClusterFewShot's performance against LabeledFewShot, BootstrapFewShot, and no optimizer (uncompiled) on both answer accuracy and retrieval sore.

In [56]:
uncompiled_baleen = SimplifiedBaleen()  # uncompiled (i.e., zero-shot) program
uncompiled_baleen_score = evaluate_on_hotpotqa(uncompiled_baleen, metric=metric)
vanilla_score = evaluate_on_hotpotqa(compiled_baleen_vanilla, metric=metric)
bootstrap_score = evaluate_on_hotpotqa(compiled_baleen_bootstrap, metric=metric)
cluster_score = evaluate_on_hotpotqa(compiled_baleen_cluster, metric=metric)

Average Metric: 26 / 50  (52.0): 100%|██████████| 50/50 [00:00<00:00, 94.97it/s]


,question,example_answer,gold_titles,context,pred_answer,answer_exact_match
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Qionghai', 'Cangzhou'}","['Cangzhou | Cangzhou () is a prefecture-level city in eastern Hebei province, People\'s Republic of China. At the 2010 census, Cangzhou\'s built-up (""or metro"") area...","No, only Cangzhou is.",
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017–18 Pittsburgh Penguins season', '2017 NHL Expansion Draft'}","[""2017 NHL Expansion Draft | The 2017 NHL Expansion Draft was an expansion draft conducted by the National Hockey League on June 18–20, 2017 to...",NHL,
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'2006–07 Detroit Red Wings season', 'Steve Yzerman'}","['Steve Yzerman | Stephen Gregory ""Steve"" Yzerman ( ; born May 9, 1965) is a Canadian retired professional ice hockey player and current general manager...",Steve Yzerman,✔️ [True]
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Collegiate Church', 'Crichton Castle'}","[""Crichton Collegiate Church | Crichton Collegiate Church is situated about 0.6 mi south west of the hamlet of Crichton in Midlothian, Scotland. Crichton itself is...",River Tyne,✔️ [True]
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Ealhswith', 'Æthelweard (son of Alfred)'}",['Beorhtnoð æthling of Kent | Beorhtnoð æthling of Kent was an English ruler of the 9-10th century AD. His father was Burgred of Mercia; his...,Æthelred I,


Average Metric: 34 / 50  (68.0): 100%|██████████| 50/50 [00:00<00:00, 87.09it/s]


,question,example_answer,gold_titles,context,pred_answer,answer_exact_match
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Qionghai', 'Cangzhou'}","['Cangzhou | Cangzhou () is a prefecture-level city in eastern Hebei province, People\'s Republic of China. At the 2010 census, Cangzhou\'s built-up (""or metro"") area...",No,✔️ [True]
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017–18 Pittsburgh Penguins season', '2017 NHL Expansion Draft'}","[""2017 NHL Expansion Draft | The 2017 NHL Expansion Draft was an expansion draft conducted by the National Hockey League on June 18–20, 2017 to...",National Hockey League,✔️ [True]
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'2006–07 Detroit Red Wings season', 'Steve Yzerman'}","['Steve Yzerman | Stephen Gregory ""Steve"" Yzerman ( ; born May 9, 1965) is a Canadian retired professional ice hockey player and current general manager...",Steve Yzerman,✔️ [True]
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Collegiate Church', 'Crichton Castle'}","[""Crichton Collegiate Church | Crichton Collegiate Church is situated about 0.6 mi south west of the hamlet of Crichton in Midlothian, Scotland. Crichton itself is...",River Tyne,✔️ [True]
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Ealhswith', 'Æthelweard (son of Alfred)'}",['Beorhtnoð æthling of Kent | Beorhtnoð æthling of Kent was an English ruler of the 9-10th century AD. His father was Burgred of Mercia; his...,Æthelred I,


Average Metric: 34 / 50  (68.0): 100%|██████████| 50/50 [00:00<00:00, 102.53it/s]


,question,example_answer,gold_titles,context,pred_answer,answer_exact_match
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Qionghai', 'Cangzhou'}","['Cangzhou | Cangzhou () is a prefecture-level city in eastern Hebei province, People\'s Republic of China. At the 2010 census, Cangzhou\'s built-up (""or metro"") area...",No,✔️ [True]
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017–18 Pittsburgh Penguins season', '2017 NHL Expansion Draft'}","[""2017 NHL Expansion Draft | The 2017 NHL Expansion Draft was an expansion draft conducted by the National Hockey League on June 18–20, 2017 to...",National Hockey League,✔️ [True]
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'2006–07 Detroit Red Wings season', 'Steve Yzerman'}","['Steve Yzerman | Stephen Gregory ""Steve"" Yzerman ( ; born May 9, 1965) is a Canadian retired professional ice hockey player and current general manager...",Steve Yzerman,✔️ [True]
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Collegiate Church', 'Crichton Castle'}","[""Crichton Collegiate Church | Crichton Collegiate Church is situated about 0.6 mi south west of the hamlet of Crichton in Midlothian, Scotland. Crichton itself is...",River Tyne,✔️ [True]
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Ealhswith', 'Æthelweard (son of Alfred)'}",['Beorhtnoð æthling of Kent | Beorhtnoð æthling of Kent was an English ruler of the 9-10th century AD. His father was Burgred of Mercia; his...,King Alfred the Great,✔️ [True]


Average Metric: 34 / 50  (68.0): 100%|██████████| 50/50 [00:00<00:00, 115.52it/s]


,question,example_answer,gold_titles,context,pred_answer,answer_exact_match
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Qionghai', 'Cangzhou'}","['Cangzhou | Cangzhou () is a prefecture-level city in eastern Hebei province, People\'s Republic of China. At the 2010 census, Cangzhou\'s built-up (""or metro"") area...",No,✔️ [True]
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017–18 Pittsburgh Penguins season', '2017 NHL Expansion Draft'}","[""2017 NHL Expansion Draft | The 2017 NHL Expansion Draft was an expansion draft conducted by the National Hockey League on June 18–20, 2017 to...",National Hockey League,✔️ [True]
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'2006–07 Detroit Red Wings season', 'Steve Yzerman'}","['Steve Yzerman | Stephen Gregory ""Steve"" Yzerman ( ; born May 9, 1965) is a Canadian retired professional ice hockey player and current general manager...",Steve Yzerman,✔️ [True]
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Collegiate Church', 'Crichton Castle'}","[""Crichton Collegiate Church | Crichton Collegiate Church is situated about 0.6 mi south west of the hamlet of Crichton in Midlothian, Scotland. Crichton itself is...",River Tyne,✔️ [True]
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Ealhswith', 'Æthelweard (son of Alfred)'}",['Beorhtnoð æthling of Kent | Beorhtnoð æthling of Kent was an English ruler of the 9-10th century AD. His father was Burgred of Mercia; his...,Æthelred I,


In [57]:
print(f"## Score for RAG: {compiled_rag_bootstrap_retrieval_score}") 
print(f"## Score for uncompiled Baleen: {uncompiled_baleen_score}")
print(f"## Score for vanilla Baleen: {vanilla_score}")
print(f"## Score for boostrap Baleen: {bootstrap_score}")
print(f"## Score for cluster Baleen: {cluster_score}")

## Score for RAG: 26.0
## Score for uncompiled Baleen: 52.0
## Score for vanilla Baleen: 68.0
## Score for boostrap Baleen: 68.0
## Score for cluster Baleen: 68.0


###### In terms of answer accuracy, all three optimizers did equally well (68.0). Now let's check their retrieval score.

In [58]:
uncompiled_baleen_retrieval_score = evaluate_on_hotpotqa(uncompiled_baleen, metric=gold_passages_retrieved)
vanilla_retrieval_score = evaluate_on_hotpotqa(compiled_baleen_vanilla, metric=gold_passages_retrieved)
bootstrap_retrieval_score = evaluate_on_hotpotqa(compiled_baleen_bootstrap, metric=gold_passages_retrieved)
cluster_retrieval_score = evaluate_on_hotpotqa(compiled_baleen_cluster, metric=gold_passages_retrieved)

print(f"## Retrieval Score for RAG: {compiled_rag_bootstrap_retrieval_score}")  # note that for RAG, compilation has no effect on the retrieval step
print(f"## Retrieval Score for uncompiled Baleen: {uncompiled_baleen_retrieval_score}")
print(f"## Retrieval Score for vanilla Baleen: {vanilla_retrieval_score}")
print(f"## Retrieval Score for boostrap Baleen: {bootstrap_retrieval_score}")
print(f"## Retrieval Score for cluster Baleen: {cluster_retrieval_score}")

Average Metric: 32 / 50  (64.0): 100%|██████████| 50/50 [00:00<00:00, 110.85it/s]


,question,example_answer,gold_titles,context,pred_answer,gold_passages_retrieved
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Qionghai', 'Cangzhou'}","['Cangzhou | Cangzhou () is a prefecture-level city in eastern Hebei province, People\'s Republic of China. At the 2010 census, Cangzhou\'s built-up (""or metro"") area...","No, only Cangzhou is.",✔️ [True]
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017–18 Pittsburgh Penguins season', '2017 NHL Expansion Draft'}","[""2017 NHL Expansion Draft | The 2017 NHL Expansion Draft was an expansion draft conducted by the National Hockey League on June 18–20, 2017 to...",NHL,✔️ [True]
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'2006–07 Detroit Red Wings season', 'Steve Yzerman'}","['Steve Yzerman | Stephen Gregory ""Steve"" Yzerman ( ; born May 9, 1965) is a Canadian retired professional ice hockey player and current general manager...",Steve Yzerman,✔️ [True]
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Collegiate Church', 'Crichton Castle'}","[""Crichton Collegiate Church | Crichton Collegiate Church is situated about 0.6 mi south west of the hamlet of Crichton in Midlothian, Scotland. Crichton itself is...",River Tyne,✔️ [True]
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Ealhswith', 'Æthelweard (son of Alfred)'}",['Beorhtnoð æthling of Kent | Beorhtnoð æthling of Kent was an English ruler of the 9-10th century AD. His father was Burgred of Mercia; his...,Æthelred I,


Average Metric: 32 / 50  (64.0): 100%|██████████| 50/50 [00:00<00:00, 105.18it/s]


,question,example_answer,gold_titles,context,pred_answer,gold_passages_retrieved
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Qionghai', 'Cangzhou'}","['Cangzhou | Cangzhou () is a prefecture-level city in eastern Hebei province, People\'s Republic of China. At the 2010 census, Cangzhou\'s built-up (""or metro"") area...",No,✔️ [True]
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017–18 Pittsburgh Penguins season', '2017 NHL Expansion Draft'}","[""2017 NHL Expansion Draft | The 2017 NHL Expansion Draft was an expansion draft conducted by the National Hockey League on June 18–20, 2017 to...",National Hockey League,✔️ [True]
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'2006–07 Detroit Red Wings season', 'Steve Yzerman'}","['Steve Yzerman | Stephen Gregory ""Steve"" Yzerman ( ; born May 9, 1965) is a Canadian retired professional ice hockey player and current general manager...",Steve Yzerman,✔️ [True]
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Collegiate Church', 'Crichton Castle'}","[""Crichton Collegiate Church | Crichton Collegiate Church is situated about 0.6 mi south west of the hamlet of Crichton in Midlothian, Scotland. Crichton itself is...",River Tyne,✔️ [True]
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Ealhswith', 'Æthelweard (son of Alfred)'}",['Beorhtnoð æthling of Kent | Beorhtnoð æthling of Kent was an English ruler of the 9-10th century AD. His father was Burgred of Mercia; his...,Æthelred I,


Average Metric: 35 / 50  (70.0): 100%|██████████| 50/50 [00:00<00:00, 102.67it/s]


,question,example_answer,gold_titles,context,pred_answer,gold_passages_retrieved
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Qionghai', 'Cangzhou'}","['Cangzhou | Cangzhou () is a prefecture-level city in eastern Hebei province, People\'s Republic of China. At the 2010 census, Cangzhou\'s built-up (""or metro"") area...",No,✔️ [True]
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017–18 Pittsburgh Penguins season', '2017 NHL Expansion Draft'}","[""2017 NHL Expansion Draft | The 2017 NHL Expansion Draft was an expansion draft conducted by the National Hockey League on June 18–20, 2017 to...",National Hockey League,✔️ [True]
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'2006–07 Detroit Red Wings season', 'Steve Yzerman'}","['Steve Yzerman | Stephen Gregory ""Steve"" Yzerman ( ; born May 9, 1965) is a Canadian retired professional ice hockey player and current general manager...",Steve Yzerman,✔️ [True]
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Collegiate Church', 'Crichton Castle'}","[""Crichton Collegiate Church | Crichton Collegiate Church is situated about 0.6 mi south west of the hamlet of Crichton in Midlothian, Scotland. Crichton itself is...",River Tyne,✔️ [True]
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Ealhswith', 'Æthelweard (son of Alfred)'}",['Beorhtnoð æthling of Kent | Beorhtnoð æthling of Kent was an English ruler of the 9-10th century AD. His father was Burgred of Mercia; his...,King Alfred the Great,


Average Metric: 32 / 50  (64.0): 100%|██████████| 50/50 [00:00<00:00, 118.29it/s]


,question,example_answer,gold_titles,context,pred_answer,gold_passages_retrieved
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Qionghai', 'Cangzhou'}","['Cangzhou | Cangzhou () is a prefecture-level city in eastern Hebei province, People\'s Republic of China. At the 2010 census, Cangzhou\'s built-up (""or metro"") area...",No,✔️ [True]
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017–18 Pittsburgh Penguins season', '2017 NHL Expansion Draft'}","[""2017 NHL Expansion Draft | The 2017 NHL Expansion Draft was an expansion draft conducted by the National Hockey League on June 18–20, 2017 to...",National Hockey League,✔️ [True]
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'2006–07 Detroit Red Wings season', 'Steve Yzerman'}","['Steve Yzerman | Stephen Gregory ""Steve"" Yzerman ( ; born May 9, 1965) is a Canadian retired professional ice hockey player and current general manager...",Steve Yzerman,✔️ [True]
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Collegiate Church', 'Crichton Castle'}","[""Crichton Collegiate Church | Crichton Collegiate Church is situated about 0.6 mi south west of the hamlet of Crichton in Midlothian, Scotland. Crichton itself is...",River Tyne,✔️ [True]
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Ealhswith', 'Æthelweard (son of Alfred)'}",['Beorhtnoð æthling of Kent | Beorhtnoð æthling of Kent was an English ruler of the 9-10th century AD. His father was Burgred of Mercia; his...,Æthelred I,


## Retrieval Score for RAG: 26.0
## Retrieval Score for uncompiled Baleen: 64.0
## Retrieval Score for vanilla Baleen: 64.0
## Retrieval Score for boostrap Baleen: 70.0
## Retrieval Score for cluster Baleen: 64.0


###### In terms of retrieval, ClusterFewShot did just as well as the LabeledFewShot optimizer and no optimizer (64.0) but BootstrapFewShot did the best (70.0)

#### Program 3: Instruction Following Program

###### Finally, let's evaluate ClusterFewShot on an instruction-following dataset (tatsulab-alpaca) which has more varied questions than HotPotQA and thus could benefit more from the clustering approach.

In [59]:
ds = load_dataset("tatsu-lab/alpaca")
ds

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 52002
    })
})

In [84]:
train = ds['train'][:50]
dev = ds['train'][50:70]

trainset = []
devset = []
for i in range(50):
    trainset.append({'question': train['instruction'][i] + " " + train['input'][i], 'answer': train['output'][i]})
for i in range(20):
    devset.append({'question': dev['instruction'][i] + " " + dev['input'][i], 'answer': dev['output'][i]})

trainset = [dspy.Example(**x).with_inputs('question') for x in trainset]
devset = [dspy.Example(**x).with_inputs('question') for x in devset]

In [70]:
class FollowInstruction(dspy.Signature):
    """Follow the instructions carefully and be succint in your response."""

    question = dspy.InputField(desc='an instruction')
    answer = dspy.OutputField()

class InstructionFollowingProgram(dspy.Module):
    def __init__(self):
        super().__init__()

        self.follow_instruction = dspy.ChainOfThought(FollowInstruction)

    def forward(self, question):
        prediction = self.follow_instruction(question=question)
        return dspy.Prediction(answer=prediction.answer)

In [85]:
vanilla_teleprompter = LabeledFewShot(k=6)
bootstrap_teleprompter = BootstrapFewShot(max_labeled_demos=6)
cluster_teleprompter = ClusterFewShot(num_labeled_demos=6)

compiled_rag_vanilla = vanilla_teleprompter.compile(InstructionFollowingProgram(), trainset=trainset)
compiled_rag_bootstrap = bootstrap_teleprompter.compile(InstructionFollowingProgram(), trainset=trainset)
compiled_rag_cluster = cluster_teleprompter.compile(InstructionFollowingProgram(), trainset=trainset)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]


  8%|▊         | 4/50 [00:00<00:00, 1003.12it/s]


Bootstrapped 4 full traces after 4 examples for up to 1 rounds, amounting to 4 attempts.


First, let's inspect the quality of the clusters that were created on this dataset.


In [25]:
for name, parameter in compiled_rag_cluster.named_predictors():
    print(name)
    cluster_assignments = parameter.cluster_assignments
    for key, val in cluster_assignments.items():
        print(key)
        for demo in val:
            print(demo)
        print()
    print()

follow_instruction
1
Example('question': Give three tips for staying healthy. , 'answer': 1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. 
2. Exercise regularly to keep your body active and strong. 
3. Get enough sleep and maintain a consistent sleep schedule.) (input_keys=['question', 'answer'])
Example('question': What are the three primary colors? , 'answer': The three primary colors are red, blue, and yellow.) (input_keys=['question', 'answer'])
Example('question': Describe the structure of an atom. , 'answer': An atom is made up of a nucleus, which contains protons and neutrons, surrounded by electrons that travel in orbits around the nucleus. The protons and neutrons have a positive charge, while the electrons have a negative charge, resulting in an overall neutral atom. The number of each particle determines the atomic number and the type of atom.) (input_keys=['question', 'answer'])
Example('question': How can we reduce air pollution? , 'answer':

###### We see that two clusters were created. The second cluster contains 5 examples which are almost all factual or historical questions, like "How did Julius Caesar die?" and "What is the capital of France". The first cluster contains all 45 other examples of more varied kinds of instructions (story telling, coding, idea generation). Ideally, we want more fine-grained clusters than this.

### Evaluation

###### Since there isn't a single "gold" answer for most of these questions, we will use human evaluation to test ClusterFewShot against LabeledFewShot and BootstrapFewShot.

In [ ]:
with open('fewshot_answers.txt', 'w', encoding='utf-8') as f:
    for i, example in enumerate(devset):
        vanilla_answer = compiled_rag_vanilla(example).answer
        bootstrap_answer = compiled_rag_bootstrap(example).answer
        cluster_answer = compiled_rag_cluster(example).answer
        f.write(f"Question {i}: {example['question']}\n")
        f.write(f"Vanilla Answer: {vanilla_answer}\n")
        f.write(f"Bootstrap Answer: {bootstrap_answer}\n")
        f.write(f"Cluster Answer: {cluster_answer}\n")
        f.write("\n")

###### From analyzing the outputs in answers.txt, we see that all the programs have more or less the same outputs on the 20 test questions. This is likely because these are questions that the underlying LM (GPT 4o-mini) is able to answer without needing demonstrations, so the affect of the bootstrapped or clustered/representative demonstrations was negligible. 

###### We shoud test these programs on a dataset or model on which high-quality demonstrations are critical for performance, in order to measure the relative impact of the different optimizers.